In [24]:
import sys
import json
import pickle
from transformers import (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

In [25]:
sys.path.append('/Users/mahbubcseju/Desktop/projects/vul_detect')

In [26]:
with open("../../data/devign/test.jsonl", 'r') as f:
    data = []
    for  line in f:
        try:
            data.append(json.loads(line.strip()))
        except Exception as e:
            print(e)


In [27]:
df = pickle.load(open('explanation_devign.pkl', 'rb'))

In [28]:
input_file = '/Users/mahbubcseju/Desktop/projects/vul_detect/data/PLBART/devign/processed/test.input0'
label_file = '/Users/mahbubcseju/Desktop/projects/vul_detect/data/PLBART/devign/processed/test.label'
with open(input_file) as inpf, open(label_file) as labelf:
        inputs = inpf.readlines()
        labels = labelf.readlines()
        print(len(inputs))

2732


In [29]:
data_lines = []
for orginal, processed, explanation in zip(data, inputs, df):
    lines = orginal['func'].split("\n")
    # print(len(lines))
    ptr = 0
    final_lines = []
    cur_line = []
    for word_ in processed.split():
        word = word_
        # print(word_[0], word_[0] == '▁')
        if len(word_) > 0 and word_[0] == '▁':
            word = word[1:]
        # print(word, ptr)
        while word not in lines[ptr]:
            final_lines.append(cur_line)
            cur_line = []
            ptr += 1
        cur_line.append(word_)
    while ptr < len(lines):
        final_lines.append(cur_line)
        cur_line = []
        ptr += 1
    
    assert len(lines) == len(final_lines)
    tokens = [tok for line in final_lines for tok in line]
    assert tokens == explanation[0]['sentence'].tokens
    data_lines.append(final_lines)

In [30]:
def get_max_match(original, start_index,  tokens):
    lo = len(original)
    lt = len(tokens)
    ind , max_score = 0, lo
    for i in range(start_index, lo-lt+ 1):
        found = True
        for j in range(lt):
            if  original[i + j] != tokens[j]:
                found = False
        if found:
            return [x for x in range(i, i + lt)]
    return []

In [31]:
def calculate_score(scores, ids):
    sum = 0
    for id in ids:
        sum += scores[id]
    return sum

In [32]:
from sklearn.preprocessing import minmax_scale


In [33]:
line_scores = []
for lines, explanation_, data_ in zip(data_lines, df, data):
    explanation = explanation_[0]['sentence']

    prev_data = list(explanation.salience)
    salience = minmax_scale(prev_data)
    scores = []
    start_index = 0
    for line in lines:
        ids = get_max_match(explanation.tokens, start_index, line)
        if ids:
            start_index = ids[-1] + 1
        scores.append(calculate_score(salience, ids))
    line_scores.append({
        'idx': data_['idx'],
        'scores': scores
    })
    

In [34]:
json.dump(line_scores, open("line_scores.json", 'w'))